In [ ]:
# tools
# view variables in notebook
%whos
# reset variables
# %reset -f

# manual search for variable definitions
# !grep -n "df =" 01_EDA.ipynb

# manual search for target_cols
# !grep -n "target_cols" 01_EDA.ipynb


You got it. Let’s make a little utility that:

* Reads a Jupyter notebook (`.ipynb`)
* Detects which section each cell belongs to by its nearest `# 2.x ...` markdown heading
* Collects **all code cells** that live under sections **2.5, 2.6, 2.7, 2.8**
* Writes them into a single `.py` file with comments separating them

---

## 🔧 Script: `aggregate_section_25_28.py`

```python
#!/usr/bin/env python3
"""
Aggregate all Python code cells from Sections 2.5–2.8 of a Jupyter notebook
into a single .py script.

Assumptions:
- Section headers are markdown cells whose FIRST line looks like:
    # 2.5 🧠 Logic Checks — Internal Consistency & Business Rules
  i.e. starts with "# 2.<something>"

Usage:
    python aggregate_section_25_28.py \
        --notebook path/to/Section2_notebook.ipynb \
        --output   resources/scripts/section_2_5_to_2_8.py
"""

import argparse
import json
from pathlib import Path
import re
from typing import Optional


SECTION_PATTERN = re.compile(r"^\s*#\s*2\.(\d+)\b")  # captures the "5" in "# 2.5 ..."


def detect_section_from_markdown(source_lines) -> Optional[int]:
    """
    Given the `source` list of a markdown cell, try to detect the Section number.
    Returns an integer like 5, 6, 7, 8 or None if not matched.
    """
    if not source_lines:
        return None

    first_line = source_lines[0]
    m = SECTION_PATTERN.match(first_line)
    if m:
        return int(m.group(1))
    return None


def aggregate_sections(notebook_path: Path, out_path: Path) -> None:
    with notebook_path.open("r", encoding="utf-8") as f:
        nb = json.load(f)

    cells = nb.get("cells", [])

    current_section: Optional[int] = None
    collected_blocks = []

    for idx, cell in enumerate(cells):
        cell_type = cell.get("cell_type")
        source = cell.get("source", [])

        # --- Track section based on markdown headers ---
        if cell_type == "markdown":
            sec = detect_section_from_markdown(source)
            if sec is not None:
                current_section = sec
            continue

        # --- Collect code cells only if in 2.5–2.8 ---
        if cell_type == "code":
            if current_section is not None and 5 <= current_section <= 8:
                # Join source (list of lines) as one block
                code_block = "".join(source)

                header_comment = (
                    f"# ======================================================================\n"
                    f"# From Section 2.{current_section} — cell index {idx}\n"
                    f"# Source notebook: {notebook_path.name}\n"
                    f"# ======================================================================\n"
                )
                collected_blocks.append(header_comment + code_block.rstrip() + "\n\n")

    # --- Write output ---
    out_path.parent.mkdir(parents=True, exist_ok=True)

    with out_path.open("w", encoding="utf-8") as f:
        if not collected_blocks:
            f.write(
                "# No cells found for Sections 2.5–2.8 in notebook "
                f"{notebook_path.name}\n"
            )
        else:
            f.write(
                "# Auto-generated aggregate of Sections 2.5–2.8 code cells\n"
                f"# Source notebook: {notebook_path.name}\n\n"
            )
            for block in collected_blocks:
                f.write(block)

    print(f"✅ Wrote {len(collected_blocks)} code blocks to: {out_path}")


def main():
    parser = argparse.ArgumentParser(
        description="Aggregate code cells from Sections 2.5–2.8 into a .py file."
    )
    parser.add_argument(
        "--notebook",
        "-n",
        type=str,
        required=True,
        help="Path to the source .ipynb notebook",
    )
    parser.add_argument(
        "--output",
        "-o",
        type=str,
        required=True,
        help="Path to the output .py file",
    )

    args = parser.parse_args()
    notebook_path = Path(args.notebook).resolve()
    out_path = Path(args.output).resolve()

    if not notebook_path.is_file():
        raise FileNotFoundError(f"Notebook not found: {notebook_path}")

    aggregate_sections(notebook_path, out_path)


if __name__ == "__main__":
    main()
```

---

## 🧪 How to use it in your Telco project

From your project root (where the `.ipynb` lives), something like:

```bash
python scripts/aggregate_section_25_28.py \
  --notebook notebooks/Section2_data_quality.ipynb \
  --output resources/scripts/section2_25_28_aggregate.py
```

That `section2_25_28_aggregate.py` becomes your **single script version** of all the logic checks / downstream sections, ready for Stage C / Stage D refactor.

If you want a variant that **also injects a small `if __name__ == "__main__"` driver** or wraps each section into a function, I can spit out that version too.


In [ ]:
# 0.0 🧩 Extract and Save All Code Cells From This Notebook
# # 0.0 Aggregate > extract > save code from notebook
from nbformat import read
from pathlib import Path

nb = read("01_EDA.ipynb", as_version=4)
code_cells = [cell.source for cell in nb.cells if cell.cell_type == "code"]
Path("aggregated_code_10-30-25.py").write_text("\n\n".join(code_cells))
print("✅ Extracted code written to aggregated_code.py")


In [ ]:
# 🧾 Inline Script (no functions)
# fully inline version that you can just drop in a cell or run as a standalone `.py` script.
# It extracts all markdown text from a Jupyter notebook and writes it into a single `.md` file.
import json
from pathlib import Path
import sys

# --- Input / output setup ---
if len(sys.argv) < 3:
    print("Usage: python extract_markdown_cells.py notebook.ipynb output.md")
    sys.exit(1)

input_path = Path(sys.argv[1])
output_path = Path(sys.argv[2])

# --- Read notebook JSON ---
if not input_path.exists():
    raise FileNotFoundError(f"❌ Notebook not found: {input_path}")
if input_path.suffix != ".ipynb":
    raise ValueError("❌ Input must be a .ipynb file")

with input_path.open("r", encoding="utf-8") as f:
    nb = json.load(f)

# --- Collect markdown cell contents ---
md_cells = []
for cell in nb.get("cells", []):
    if cell.get("cell_type") == "markdown":
        text = "".join(cell.get("source", []))
        md_cells.append(text.strip())

# --- Write combined markdown file ---
if md_cells:
    output_path.parent.mkdir(parents=True, exist_ok=True)
    combined = "\n\n---\n\n".join(md_cells)
    with output_path.open("w", encoding="utf-8") as f:
        f.write(combined)
    print(f"✅ Extracted {len(md_cells)} markdown cells → {output_path}")
else:
    print("⚠️ No markdown cells found.")

### 🪄 Usage example
# python extract_markdown_cells.py Level_3_Telco.ipynb Level_3_Telco_Markdown.md


In [ ]:
### 🧠 Optional — Run inside a notebook instead of CLI
# If you just want to run it **inside Jupyter** (no command-line args), simplify further:
# That version is **function-free**, **flat**, and ready to paste into your Telco project’s tooling notebook or terminal script.

import json
from pathlib import Path
from datetime import datetime

input_path = Path(".Level_3/notebooks/01_EDA.ipynb")
output_path = Path(f"{datetime.now().strftime('%m%d_%Y_%H%M%S')}_Agg_01_EDA_Telco_Markdown.md")

nb = json.loads(Path(input_path).read_text(encoding="utf-8"))
md_cells = [ "".join(c["source"]).strip() for c in nb["cells"] if c["cell_type"] == "markdown" ]

combined = "\n\n---\n\n".join(md_cells)
Path(output_path).write_text(combined, encoding="utf-8")


print(f"✅ Wrote {len(md_cells)} markdown cells to {output_path}")


In [ ]:
# Inline Aggregate of markdown cells and code comments from a Jupyter notebook
import json
from pathlib import Path
from datetime import datetime

input_path = Path("01_EDA.ipynb")
output_path = Path(f"_Agg_md_01_EDA_Telco_{datetime.now().strftime('%m-%d-%Y_%H%M')}.md")

nb = json.loads(input_path.read_text(encoding="utf-8"))
md_blocks = []

for cell in nb["cells"]:
    if cell["cell_type"] == "markdown":
        md_blocks.append("".join(cell["source"]).strip())
    elif cell["cell_type"] == "code":
        lines = cell["source"]
        top_comments = []
        for line in lines:
            if line.strip().startswith("#"):
                top_comments.append(line.lstrip("# ").rstrip())
            elif line.strip() == "":
                top_comments.append("")  # allow spacing within header block
            else:
                break  # stop at first non-comment line
        if top_comments:
            md_blocks.append("\n".join(top_comments).strip())

combined = "\n\n---\n\n".join(md_blocks)
Path(output_path).write_text(combined, encoding="utf-8")

print(f"✅ Wrote {len(md_blocks)} markdown or comment blocks → {output_path}")



In [ ]:
# Here’s a compact Python script that walks a Jupyter notebook and extracts, for each cell:

# 1. `document.title = "..."` (inside an HTML `<script>` in a **markdown** cell), or
# 2. the **first Markdown `#` heading** line in a **markdown** cell, or
# 3. the **first `#` comment** line in a **code** cell.

# It prints a clean outline and also saves a CSV.

#!/usr/bin/env python3
"""
Extract titles from a Jupyter notebook, using this precedence:
  1) From <script> ... document.title="..." ... </script> in a markdown cell
  2) First Markdown heading line (# ... ) in a markdown cell
  3) First comment line (# ... ) in a code cell
Outputs to stdout and writes Notebook_Titles.csv
"""

import json, re, csv
from pathlib import Path

# --- CONFIG
input_path = Path("01_EDA.ipynb")
output_csv = Path("./report/Notebook_Titles.csv")

nb = json.loads(input_path.read_text(encoding="utf-8"))

# Regexes
re_doc_title = re.compile(r'document\.title\s*=\s*([\'"])(.*?)\1', re.IGNORECASE | re.DOTALL)
re_html_title = re.compile(r'<title[^>]*>(.*?)</title>', re.IGNORECASE | re.DOTALL)
re_md_heading = re.compile(r'^\s{0,3}#{1,6}\s+(.*\S)\s*$', re.MULTILINE)
re_code_comment = re.compile(r'^\s*#\s*(.*\S)\s*$', re.MULTILINE)

rows = []
for i, cell in enumerate(nb.get("cells", []), start=1):
    ctype = cell.get("cell_type")
    src = "".join(cell.get("source", []))
    title = None
    source_kind = None

    if ctype == "markdown":
        # 1) document.title in <script>...</script>
        m = re_doc_title.search(src)
        if m:
            title = m.group(2).strip()
            source_kind = "md:script document.title"
        else:
            # 1b) <title>...</title>
            m = re_html_title.search(src)
            if m:
                title = re.sub(r'\s+', ' ', m.group(1)).strip()
                source_kind = "md:<title>"
            else:
                # 2) first Markdown # heading
                m = re_md_heading.search(src)
                if m:
                    title = m.group(1).strip()
                    source_kind = "md:# heading"

    elif ctype == "code":
        # 3) first comment line
        m = re_code_comment.search(src)
        if m:
            title = m.group(1).strip()
            source_kind = "code:# comment"

    if title:
        rows.append({"cell_index": i, "cell_type": ctype, "source": source_kind, "title": title})

# --- Print a readable outline
for r in rows:
    print(f"[{r['cell_index']:>3}] ({r['cell_type']:<8} | {r['source']:<22})  {r['title']}")

# --- Save CSV
if rows:
    with output_csv.open("w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["cell_index","cell_type","source","title"])
        w.writeheader()
        w.writerows(rows)
    print(f"\n✅ Wrote {len(rows)} titles → {output_csv}")
else:
    print("ℹ️ No titles/headings/comments matched the rules.")


# **Notes**

# * It prefers `document.title="..."` if present, then `<title>...</title>`, then Markdown `# ...`, then code `# ...`.
# * It collapses whitespace inside `<title>` tags.
# * Adjust `input_path` to your notebook file name.
